In [1]:
import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator

from net import Net
from aco import ACO
from utils import *

torch.manual_seed(1234)

EPS = 1e-10
device = 'cpu'

max_len = {
    20: 2,
    100: 4,
    500: 9
}

In [2]:
def infer_instance(model, instance, k_sparse, n_ants, t_aco_diff):
    pyg_data, distances, prizes = instance
    if model:
        model.eval()
        heu_mat = model.reshape(pyg_data, model(pyg_data)) + EPS
        aco = ACO(distances, prizes, max_len[len(prizes)], n_ants, heuristic=heu_mat, device=device)
    else:
        aco = ACO(distances, prizes, max_len[len(prizes)], n_ants, device=device)
        aco.sparsify(k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost, _ = aco.run(t)
        results[i] = best_cost
    return results
    
@torch.no_grad()
def test(dataset, model, n_ants, k_sparse, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in dataset:
        results = infer_instance(model, instance, k_sparse, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on OP20

MetaACO

In [3]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net = Net().to(device)
net.load_state_dict(torch.load(f'../pretrained/op/op{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  467.0962164402008
T=1, average obj. is 5.132494926452637.
T=10, average obj. is 5.285694599151611.
T=20, average obj. is 5.326320171356201.
T=30, average obj. is 5.359354019165039.
T=40, average obj. is 5.36385440826416.
T=50, average obj. is 5.368360996246338.
T=100, average obj. is 5.37666130065918.


In [9]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
avg_aco_best, duration = test(test_list, None, n_ants, k_sparse, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  396.59278535842896
T=1, average obj. is 4.329096794128418.
T=10, average obj. is 4.939450263977051.
T=20, average obj. is 5.068620681762695.
T=30, average obj. is 5.116286754608154.
T=40, average obj. is 5.151432991027832.
T=50, average obj. is 5.2041521072387695.
T=100, average obj. is 5.2815937995910645.
